# Swim Transformer : Hierarchical Vision Transformer using Shifted Windows

在2021年微软亚洲研究院基于Transformer架构提出了一种通用的视觉backnone，使用分层窗口注意力机制移位窗口注意力计算等方法解决了原有的Transformer因为模型庞大难以应用在目标检测和语义分割这种像素级任务上的缺陷。

目前Transformer应用到图像领域主要有两大挑战：

- 视觉实体变化大，在不同场景下视觉Transformer性能未必很好
- 图像分辨率高，像素点多，Transformer基于全局自注意力的计算导致计算量较大

针对上述两个问题，我们提出了一种**包含滑窗操作，具有层级设计**的Swin Transformer。

其中滑窗操作包括**不重叠的local window，和重叠的cross-window**。将注意力计算限制在一个窗口中，**一方面能引入CNN卷积操作的局部性，另一方面能节省计算量。**

## 模型架构


<div align = "center">
    <img src = ./images/swim-tfm-0.png width = 90%>
</div>

整个模型采取层次化的设计，一共包含4个Stage，每个stage都会缩小输入特征图的分辨率，翻倍通道数，像CNN一样逐层扩大感受野。

与Vit相比，Swim-tfm有以下几点区别：

- 在模型开头取消Position embedding和Class token的操作，直接进行patch分割
- 从Stage2开始，最开始都进行Patch Merging，用于降低图片分辨率
- MSA修改为Window MSA以及Shifted window MSA，且先经过W-MSA再经过SW-MSA

参考[微软官方代码](https://github.com/microsoft/Swin-Transformer)，下面是Swim-tfm实现的总体流程。

```python
class SwinTransformer(nn.Module):
    def __init__(...):
        super().__init__()
        ...
        # absolute position embedding
        if self.ape:
            self.absolute_pos_embed = nn.Parameter(torch.zeros(1, num_patches, embed_dim))
            
        self.pos_drop = nn.Dropout(p=drop_rate)

        # build layers
        self.layers = nn.ModuleList()
        for i_layer in range(self.num_layers):
            layer = BasicLayer(...)
            self.layers.append(layer)

        self.norm = norm_layer(self.num_features)
        self.avgpool = nn.AdaptiveAvgPool1d(1)
        self.head = nn.Linear(self.num_features, num_classes) if num_classes > 0 else nn.Identity()

    def forward_features(self, x):
        x = self.patch_embed(x)
        if self.ape:
            x = x + self.absolute_pos_embed
        x = self.pos_drop(x)

        for layer in self.layers:
            x = layer(x)

        x = self.norm(x)  # B L C
        x = self.avgpool(x.transpose(1, 2))  # B C 1
        x = torch.flatten(x, 1)
        return x

    def forward(self, x):
        x = self.forward_features(x)
        x = self.head(x)
        return x
```

接下来再来看各个组件的实现细节

## Patch Embedding

在整个Swim-tfm的流程中，第一步就是划分图像为数个Patch，之后再嵌入向量。

具体做法为使用 **2d卷积** 对图像进行patch划分，然后进行Flatten操作进行向量嵌入。

In [11]:
import torch
import torch.nn as nn

def to_tuple(t):
    return t if isinstance(t, tuple) else (t, t)


class Patch_Embedding(nn.Module):
    def __init__(self, image_size, patch_size, in_channels, embedding_dim, norm_layer=None):
        super(Patch_Embedding, self).__init__()
        image_size = to_tuple(image_size)
        patch_size = to_tuple(patch_size)
        patch_number = (image_size[0] // patch_size[0]) * (image_size[1] // patch_size[1])
        self.image_size = image_size
        self.patch_size = patch_size
        self.patch_number = patch_number

        self.in_channels = in_channels
        self.embedding_dim = embedding_dim

        self.projection = nn.Conv2d(in_channels, embedding_dim, kernel_size=patch_size, stride=patch_size)
        if norm_layer is not None:
            self.norm_layer = norm_layer(embedding_dim)
        else:
            self.norm_layer = None

    def forward(self, x):
        x = self.projection(x)
        print("dim after projection : {} ".format(x.shape))
        x = torch.flatten(x, 2) # n c h w -> n c h*w
        x = torch.transpose(x, 1, 2) # n c h*c -> n h*w c
        if self.norm_layer is not None:
            x = self.norm_layer(x)
        return x


if __name__ == "__main__":
    PatchEmbedding = Patch_Embedding(image_size=224,
                                     patch_size=4,
                                     in_channels=3,
                                     embedding_dim=96)
    
    img = torch.randn(1, 3, 224, 224)

    result = PatchEmbedding(img)
    print(result.shape)

dim after projection : torch.Size([1, 96, 56, 56]) 
torch.Size([1, 3136, 96])


## Patch Merging

该模块的作用是在每个Stage开始前做降采样，用于缩小分辨率，调整通道数 进而形成层次化的设计，同时也能节省一定运算量。

> 在CNN中，则是在每个Stage开始前用stride=2的卷积/池化层来降低分辨率。
> 每次降采样是两倍，因此在行方向和列方向上，间隔2选取元素。

然后拼接在一起作为一整个张量，最后展开。**此时通道维度会变成原先的4倍（因为H,W各缩小2倍），此时再通过一个全连接层再调整通道维度为原来的两倍**

In [14]:
class Patch_Merging(nn.Module):
    def __init__(self, input_resolution, dim, norm_layer=None):
        super(Patch_Merging, self).__init__()
        self.input_resolution = input_resolution
        self.dim = dim
        self.reduction = nn.Linear(4 * dim, 2 * dim, bias=False)
        if norm_layer is not None:
            self.norm_layer = norm_layer
        else:
            self.norm_layer = nn.LayerNorm(4 * dim)

    def forward(self, x):
        '''
        x : b h*w c
        '''
        H, W  = self.input_resolution
        B, L, C = x.shape
        assert H*W == L, "input features has wrong size."
        assert H % 2 == 0 and W % 2 == 0, f"x size ({H}*{W}) are not even."

        x = x.view(B, H, W, C)

        x0 = x[:, 0::2, 0::2, :] # b, h/2, w/2, c
        x1 = x[:, 1::2, 0::2, :]
        x2 = x[:, 0::2, 1::2, :]
        x3 = x[:, 1::2, 1::2, :]

        x = torch.cat([x0, x1, x2, x3], -1) # b, h/2, w/2, 4*c
        x = self.norm_layer(x)
        x = self.reduction(x)
        return x


if __name__ == "__main__":
    # middle_feature_map = torch.randn(1, 3136, 96)

    PatchMerging = Patch_Merging(input_resolution=[56, 56],
                                 dim=96)

    result = PatchMerging(result)
    print(result.shape)

torch.Size([1, 28, 28, 192])


上述流程示意图如下：

<div align = "center">
    <img src = ./images/swim-tfm-1.png width = 90%>
</div>